# First Order Motion Model for Face Rearrangement (suited for deepfake memes)

Original code made my Aliaksandr Siarohin, edited by bemxio. For a tutorial on how to use this, head over to https://drive.google.com/file/d/1BnHOZFklXe9KzL4jQt0mavvNH4eL28bJ/view?usp=drivesdk

**Clone original repository from GitHub**

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 249 (delta 0), reused 0 (delta 0), pack-reused 246
Receiving objects: 100% (249/249), 72.13 MiB | 10.94 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [2]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**Load driving video and source image**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
 
warnings.filterwarnings('ignore')
source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/02.png')
 
reader = imageio.get_reader('/content/gdrive/My Drive/first-order-motion-model/04.mp4')
driving_video = [frame for frame in reader]
 
# get FPS of a driving video,
# to make the generated video have the right speed
video_fps = reader.get_meta_data()['fps']
 
# resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
 
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
 
HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [5]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

**Perform image animation**

In [6]:
from demo import make_animation
from skimage import img_as_ubyte
 
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)
 
# save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=video_fps)
# video can be downloaded from /content folder
 
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 990/990 [00:44<00:00, 22.02it/s]


**Add audio from the template video to the generated one**

In [7]:
from moviepy.editor import VideoFileClip, AudioFileClip
import os
 
video = VideoFileClip('../generated.mp4')
audio = AudioFileClip('/content/gdrive/My Drive/first-order-motion-model/04.mp4')
 
video = video.set_audio(audio)
os.remove('../generated.mp4')
video.write_videofile('../generated.mp4')

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)4096000/45929032 bytes (8.9%)8323072/45929032 bytes (18.1%)12607488/45929032 bytes (27.4%)16891904/45929032 bytes (36.8%)21200896/45929032 bytes (46.2%)25477120/45929032 bytes (55.5%)29171712/45929032 bytes (63.5%)33472512/45929032 bytes (72.9%)37363712/45929032 bytes (81.4%)41648128/45929032 bytes (90.7%)45883392/45929032 bytes (99.9%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

100%|██████████| 728/728 [00:00<00:00, 801.95it/s]

[MoviePy] Done.
[MoviePy] Writing video ../generated.mp4



100%|█████████▉| 990/991 [00:03<00:00, 267.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../generated.mp4 

